### Appendix: LSTM with Attention
This model we unfortunatly where not able to get to work properly in the given timeframe. We still included it for completeness, if we had more time we would try to understand it more and get it to predict properly. Initial results look promising even though the performance is not as good as the other models presented so far. 
We also faced some hardware issues (mainly vRAM) which made it unfeasable to continue with the model for now.

In [2]:
import pandas as pd
import math
import keras
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import save_model
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters
from pickle import dump,load
import warnings
warnings.simplefilter("ignore", UserWarning)
from tensorflow.keras.layers import Normalization
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.layers.core import Permute
from keras.layers import Input, Dense, Activation, RepeatVector, add ,Flatten, TimeDistributed, Multiply
from keras.layers import Embedding, LSTM, BatchNormalization, dot, concatenate
from keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
train = pd.read_csv("data/pooled_train_300.csv")

In [5]:
# train.insert(0, "open_time ", pd.to_datetime(train["index"]))
train.sort_values(by='index', inplace = True)

#keep train where train zero > 50%
train["zeros_perc"] = (train == 0).astype(int).sum(axis=1).div(train.shape[1]-4)
train = train[train["zeros_perc"]< 0.3]

# train, val= ()
train_1, val_train_1 = train_test_split(train, test_size=0.2, random_state=42, shuffle = False)

In [6]:
y_train = train_1["returns"]
X_train = train_1.drop(["index", "asset", "returns", "Unnamed: 0", "zeros_perc"], axis=1)

# val train: 
y_val = val_train_1["returns"]
X_val = val_train_1.drop(["index", "asset", "returns", "Unnamed: 0", "zeros_perc"], axis=1)


# y_test = test["returns"]
# X_test = test.drop(["index", "asset", "returns", "Unnamed: 0"], axis=1)

In [7]:
X_train= X_train.to_numpy().reshape(X_train.shape[0],X_train.shape[1],1)
y_train= y_train.to_numpy().reshape(y_train.shape[0],1)

X_val_train= X_val.to_numpy().reshape(X_val.shape[0],X_val.shape[1],1)
y_val_train= y_val.to_numpy().reshape(y_val.shape[0],1)

In [26]:
#SOURCE: https://machinelearningmastery.com/use-dropout-lstm-networks-time-series-forecasting/
#INPUT layer
n_hidden = 400
input_train = Input(shape=(X_train.shape[1], X_train.shape[2]))
output_train = Input(shape=(y_train.shape[1], 1))

# ENCODER LSTM (returns last hidden and last cell state and (new due to attention alignment score))
encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
    n_hidden, activation='tanh', dropout=0.0, recurrent_dropout=0, 
    return_state=True, return_sequences=True)(input_train)
print(encoder_stack_h)
print(encoder_last_h)
print(encoder_last_c)
### add batchnorm to avoid gradient explosion
encoder_last_h = BatchNormalization(momentum=0.4)(encoder_last_h)
encoder_last_c = BatchNormalization(momentum=0.4)(encoder_last_c)

# DEDODER LSTM
decoder_input = RepeatVector(output_train.shape[1])(encoder_last_h)
print(decoder_input)
### alignement score
####stacked hidden state
decoder_stack_h = LSTM(n_hidden, activation='tanh', dropout=0.0, recurrent_dropout=0,
 return_state=False, return_sequences=True)(
 decoder_input, initial_state=[encoder_last_h, encoder_last_c])
print(decoder_stack_h)

# ATTENTION layer
attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
attention = Activation('softmax')(attention)
print(attention)
### context vector and batch normalization
context = dot([attention, encoder_stack_h], axes=[2,1])
context = BatchNormalization(momentum=0.4)(context)
print(context)
### concat context and stacked hidden states
decoder_combined_context = concatenate([context, decoder_stack_h])
print(decoder_combined_context)
out = TimeDistributed(Dense(output_train.shape[2]))(decoder_combined_context)
print(out)

# COMPILE
model = Model(inputs=input_train, outputs=out)
opt = tf.keras.optimizers.Adam(lr=0.001, clipnorm=1)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae'])
model.summary()

KerasTensor(type_spec=TensorSpec(shape=(None, 300, 400), dtype=tf.float32, name=None), name='lstm_16/PartitionedCall:1', description="created by layer 'lstm_16'")
KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name=None), name='lstm_16/PartitionedCall:2', description="created by layer 'lstm_16'")
KerasTensor(type_spec=TensorSpec(shape=(None, 400), dtype=tf.float32, name=None), name='lstm_16/PartitionedCall:3', description="created by layer 'lstm_16'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 400), dtype=tf.float32, name=None), name='repeat_vector_8/Tile:0', description="created by layer 'repeat_vector_8'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 400), dtype=tf.float32, name=None), name='lstm_17/PartitionedCall:1', description="created by layer 'lstm_17'")
KerasTensor(type_spec=TensorSpec(shape=(None, 1, 300), dtype=tf.float32, name=None), name='activation_8/Softmax:0', description="created by layer 'activation_8'")
KerasTensor(type_spec=TensorSpe

In [28]:
epc = 100
es = EarlyStopping(monitor='val_loss', mode='min', patience=3)
history = model.fit(X_train[:, :], y_train[:, :], validation_split=0.2, 
                    epochs=epc, verbose=1, callbacks=[es], 
                    batch_size=1200)
train_mae = history.history['mae']
valid_mae = history.history['val_mae']
 
model.save('model_forecasting_seq2seq7.h5')

Epoch 1/100
200/200 [==============================] - 146s 721ms/step - loss: 0.0044 - mae: 0.0092 - val_loss: 3.3158e-04 - val_mae: 0.0156
Epoch 2/100
200/200 [==============================] - 142s 712ms/step - loss: 3.1909e-04 - mae: 0.0077 - val_loss: 1.3832e-04 - val_mae: 0.0078
Epoch 3/100
200/200 [==============================] - 143s 713ms/step - loss: 1.6820e-04 - mae: 0.0076 - val_loss: 1.1691e-04 - val_mae: 0.0063
Epoch 4/100
200/200 [==============================] - 143s 713ms/step - loss: 1.6907e-04 - mae: 0.0076 - val_loss: 2.4708e-04 - val_mae: 0.0128
Epoch 5/100
200/200 [==============================] - 142s 712ms/step - loss: 1.7243e-04 - mae: 0.0077 - val_loss: 1.2101e-04 - val_mae: 0.0066
Epoch 6/100
200/200 [==============================] - 142s 712ms/step - loss: 1.6907e-04 - mae: 0.0077 - val_loss: 1.1600e-04 - val_mae: 0.0062
Epoch 7/100
200/200 [==============================] - 142s 711ms/step - loss: 1.6815e-04 - mae: 0.0076 - val_loss: 1.1582e-04 - val_m

Our working notes so far: